In [4]:
from package.get_tools import get_tools

In [8]:
tools = await get_tools()

In [7]:
tools

[Tool(name='add', description='Add\n\nAdd two numbers\n\nArgs:\n    a (int): first number\n    b (int): second number\n    \nReturns:\n    dict: a dictionary with the sum\n\n### Responses:\n\n**200**: Successful Response (Success Response)\nContent-Type: application/json\n\n**Output Schema:**\n```json\n{\n  "type": "object",\n  "additionalProperties": true,\n  "title": "Response Add"\n}\n```\n**422**: Validation Error\nContent-Type: application/json\n\n**Example Response:**\n```json\n{\n  "detail": [\n    {\n      "loc": [],\n      "msg": "Message",\n      "type": "Error Type"\n    }\n  ]\n}\n```\n\n**Output Schema:**\n```json\n{\n  "properties": {\n    "detail": {\n      "items": {\n        "properties": {\n          "loc": {\n            "items": {},\n            "type": "array",\n            "title": "Location"\n          },\n          "msg": {\n            "type": "string",\n            "title": "Message"\n          },\n          "type": {\n            "type": "string",\n          

In [20]:
from broai.llm_management.ollama import BedrockOllamaChat
import json
model = BedrockOllamaChat()

In [9]:
tools[0]

Tool(name='add', description='Add\n\nAdd two numbers\n\nArgs:\n    a (int): first number\n    b (int): second number\n    \nReturns:\n    dict: a dictionary with the sum\n\n### Responses:\n\n**200**: Successful Response (Success Response)\nContent-Type: application/json\n\n**Output Schema:**\n```json\n{\n  "type": "object",\n  "additionalProperties": true,\n  "title": "Response Add"\n}\n```\n**422**: Validation Error\nContent-Type: application/json\n\n**Example Response:**\n```json\n{\n  "detail": [\n    {\n      "loc": [],\n      "msg": "Message",\n      "type": "Error Type"\n    }\n  ]\n}\n```\n\n**Output Schema:**\n```json\n{\n  "properties": {\n    "detail": {\n      "items": {\n        "properties": {\n          "loc": {\n            "items": {},\n            "type": "array",\n            "title": "Location"\n          },\n          "msg": {\n            "type": "string",\n            "title": "Message"\n          },\n          "type": {\n            "type": "string",\n           

In [203]:
from typing import Annotated, _AnnotatedAlias, _LiteralGenericAlias
from typing import get_type_hints, get_args, get_origin

def map_format(annotation):
    _format = annotation
    if _format is int:
        return "integer"
    if _format is str:
        return "string"
    if _format is bool:
        return "boolean"
    if _format is float:
        return "number"
    if isinstance(_format, _LiteralGenericAlias):
        _args = _format.__args__[0]
        if isinstance(_args, str):
            return "string"
        if isinstance(_args, int):
            return "integer"
    raise ValueError(f"Unsupported type: {_format}")

def get_properties(annotations:dict) -> dict:
    properties = {}
    for param, anno in annotations.items():
        if isinstance(anno, _AnnotatedAlias):
            _type = map_format(get_args(anno)[0])
            _description = get_args(anno)[1]
            _param = {
                "type": _type, 
                "description": _description,
            }
            if isinstance(get_args(anno)[0], _LiteralGenericAlias):
                _param["enum"] = list(get_args(get_args(anno)[0]))
                
            properties[param] = _param

    return properties

def tool_request(func):
    annotations = func.__annotations__
    defaults = func.__defaults__ or []
    properties = get_properties(annotations)
    n_params = len(properties)
    required = list(properties.keys())[0:n_params-len(defaults)]
    return {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": func.__doc__,
            "parameters": {
                "type": "object",
                "properties": properties
            },
        "required": required
        }
    }

def tool_response(tool_prompt:dict):
    name = tool_prompt["function"]["name"]
    properties = tool_prompt["function"]["parameters"]["properties"]
    arguments = {}
    for param, property in properties.items():
        arguments[param] = property['description']
    return {
        "function": {
            "name": name,
            "arguments": arguments
        }
    }

In [204]:
from typing import Annotated, _AnnotatedAlias, Literal

def test_function(a: Annotated[int, "first integer"], 
                  b: Annotated[float, "second float"],
                  c: Annotated[str, "third string"] = "default",
                  d: Annotated[Literal["optional", "value"], "forth literal"] = "optional_value"
                  ) -> int:
    """This is a test function"""
    return a+b

# dir(test_function)

In [205]:
import json
tool_prompt = tool_request(test_function)
print(json.dumps(tool_prompt, indent=2))

{
  "type": "function",
  "function": {
    "name": "test_function",
    "description": "This is a test function",
    "parameters": {
      "type": "object",
      "properties": {
        "a": {
          "type": "integer",
          "description": "first integer"
        },
        "b": {
          "type": "number",
          "description": "second float"
        },
        "c": {
          "type": "string",
          "description": "third string"
        },
        "d": {
          "type": "string",
          "description": "forth literal",
          "enum": [
            "optional",
            "value"
          ]
        }
      }
    },
    "required": [
      "a",
      "b"
    ]
  }
}


In [206]:
tool_response(tool_prompt)

{'function': {'name': 'test_function',
  'arguments': {'a': 'first integer',
   'b': 'second float',
   'c': 'third string',
   'd': 'forth literal'}}}

In [136]:
test_function.__name__

'test_function'

In [137]:
test_function.__doc__

'This is a test function'

In [173]:
d = test_function.__annotations__['d']
d

typing.Annotated[typing.Literal['optional', 'value'], 'this is a test']

In [178]:
get_args(get_args(d)[0])

('optional', 'value')

In [102]:
[1,2,3][:0]

[]

In [51]:
test_function.__defaults__

(2, 'default')

In [ ]:
tool = tools[0]
print(tool.description)

Add

Add two numbers

Args:
    a (int): first number
    b (int): second number
    
Returns:
    dict: a dictionary with the sum

### Responses:

**200**: Successful Response (Success Response)
Content-Type: application/json

**Output Schema:**
```json
{
  "type": "object",
  "additionalProperties": true,
  "title": "Response Add"
}
```
**422**: Validation Error
Content-Type: application/json

**Example Response:**
```json
{
  "detail": [
    {
      "loc": [],
      "msg": "Message",
      "type": "Error Type"
    }
  ]
}
```

**Output Schema:**
```json
{
  "properties": {
    "detail": {
      "items": {
        "properties": {
          "loc": {
            "items": {},
            "type": "array",
            "title": "Location"
          },
          "msg": {
            "type": "string",
            "title": "Message"
          },
          "type": {
            "type": "string",
            "title": "Error Type"
          }
        },
        "type": "object",
        "require

In [19]:
model.run(
    system_prompt="You are a helpful assistant.",
    messages=[
    model.UserMessage("Hi there")
])

'How can I assist you today?'